# © Artur Czarnecki. All rights reserved.
# Integrax framework - proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [ ]:
from typing import Any, Optional
from intergrax.llm_adapters.llm_provider import LLMAdapterRegistry, LLMProvider
from intergrax.llm_adapters.llm_usage_track import LLMUsageTracker
from intergrax.memory.conversational_memory import ConversationalMemory
from intergrax.tools.tools_agent import ToolsAgent, ToolsAgentConfig
from intergrax.tools.tools_base import ToolRegistry, ToolBase
from pydantic import BaseModel, Field

# ---------- WEATHER TOOL ----------
class WeatherArgs(BaseModel):
    """
    Argument schema for the weather tool.

    Fields:
      - city: name of the city for which we want to fetch weather information.
    """
    city: str = Field(..., description="City name, e.g. 'Warsaw'")


class WeatherTool(ToolBase):
    """
    Simple demo weather tool.

    Responsibilities:
      - Accept a city name.
      - Return mock current weather information for that city.

    NOTE:
      In a real implementation this would call an external weather API.
    """
    name = "get_weather"
    description = (
        "Returns current weather for a city. "
        "Use only for queries about weather/temperature/conditions in a city."
    )
    schema_model = WeatherArgs

    def run(self, 
            run_id:Optional[str] = None,
            llm_usage_tracker: Optional[LLMUsageTracker] = None,
            **kwargs) -> Any:
        city = kwargs["city"]
        # Demo output (static, not calling any real API).
        return {"city": city, "tempC": 12.3, "summary": "partly cloudy"}


# ---------- CALCULATOR TOOL ----------
class CalcArgs(BaseModel):
    """
    Argument schema for the calculator tool.

    Fields:
      - expression: a simple arithmetic expression with numbers and operators.
    """
    expression: str = Field(
        ...,
        description=(
            "A safe arithmetic expression to evaluate, "
            "e.g. '235*17', '2*(3+4)'. No variables."
        ),
    )


class CalcTool(ToolBase):
    """
    Basic arithmetic calculator tool.

    Responsibilities:
      - Validate a user-provided expression.
      - Evaluate it using a restricted environment.
      - Return both the expression and numeric result.
    """
    name = "calc_expression"
    description = (
        "Safely evaluates a basic arithmetic expression (integers/floats, + - * / parentheses). "
        "Use for math/calculation questions."
    )
    schema_model = CalcArgs

    def run(self, 
            run_id:Optional[str] = None,
            llm_usage_tracker: Optional[LLMUsageTracker] = None,
            **kwargs) -> Any:
        expr = kwargs["expression"]

        # Minimal, safe evaluation – only allow basic numeric and operator characters.
        # This is meant purely as a sandboxed arithmetic evaluator, not a general 'eval'.
        allowed = "0123456789.+-*/() "
        if not all(ch in allowed for ch in expr):
            raise ValueError("Unsupported characters in expression.")

        # Perform the calculation in a restricted environment.
        try:
            result = eval(expr, {"__builtins__": {}}, {})
        except Exception as e:
            raise ValueError(f"Invalid expression: {e}")

        return {"expression": expr, "result": result}


# ---------- AGENT SETUP ----------

# Conversational memory shared across interactions.
# Used by the tools agent to keep track of dialogue context.
memory = ConversationalMemory()

# Registry holding all available tools for the agent.
tools = ToolRegistry()
tools.register(WeatherTool())
tools.register(CalcTool())

# LLM used as the planner/controller for tools.
# Here we use an Ollama-backed model; the agent will:
#   - interpret user questions,
#   - decide whether tools are needed,
#   - call tools via the registry,
#   - integrate tool outputs into a final answer.
llm = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

# Tools agent that orchestrates:
#   - LLM reasoning,
#   - tool selection and invocation,
#   - conversational memory updates.
agent = ToolsAgent(
    llm=llm,
    tools=tools,
    memory=memory,
    config=ToolsAgentConfig(),
    verbose=True,
)


# ---------- TEST 1: weather → should select get_weather ----------
res1 = agent.run(
    input_data="What is the weather and temperature in Warsaw ?", 
    context=None)
print("[ANS 1]", res1["answer"])
print("[TOOLS 1]", res1["tool_traces"])

# ---------- TEST 2: calculation → should select calc_expression ----------
res2 = agent.run(
    input_data="Calculate 235*17 and return result without markdowns and comments.", 
    context=None)
print("[ANS 2]", res2["answer"])
print("[TOOLS 2]", res2["tool_traces"])

[intergraxToolsAgent] Iteration 1 (planner)
[intergraxToolsAgent] Calling tool: get_weather({'city': 'Warsaw'})
[intergraxToolsAgent] Iteration 2 (planner)
[ANS 1] The current weather in Warsaw is partly cloudy with a temperature of 12.3 degrees Celsius.
[TOOLS 1] [{'tool': 'get_weather', 'args': {'city': 'Warsaw'}, 'output_preview': '{"city": "Warsaw", "tempC": 12.3, "summary": "partly cloudy"}', 'output': {'city': 'Warsaw', 'tempC': 12.3, 'summary': 'partly cloudy'}}]
[intergraxToolsAgent] Iteration 1 (planner)
[intergraxToolsAgent] Calling tool: calc_expression({'expression': '235*17'})
[intergraxToolsAgent] Iteration 2 (planner)
[ANS 2] 3995
[TOOLS 2] [{'tool': 'calc_expression', 'args': {'expression': '235*17'}, 'output_preview': '{"expression": "235*17", "result": 3995}', 'output': {'expression': '235*17', 'result': 3995}}]


In [3]:
agent.llm.export_run_stats_dict()

{'run_id': 'general',
 'calls': 4,
 'input_tokens': 1410,
 'output_tokens': 78,
 'total_tokens': 1488,
 'duration_ms': 3989,
 'errors': 0}